In [1]:
from MAUtil import *
from MAInit import *
from MAPost import *
np.set_printoptions(linewidth=200)
np.set_printoptions(threshold=5000)

In [2]:
# fingerprint_adslab_multiads(atoms)

# Note:By removing molecule consisted of two atoms and adsorbed Kripton, atoms index has changed!!

In [3]:
# env = 'local'
env = 'spacom'
name = 'Au_111_u2_no021_CO_n4_d3.traj'
name = 'Au_111_u2_no006_CO_n4_d3.traj'
name = 'Au_111_u2_no004_CO_n4_d3.traj'
name = 'Au_111_u2_no030_CO_n4_d3.traj'
# name = 'Au_111_u2_no035_CO_n4_d3.traj'
# name = 'Cu_100_u2_no13_CO_n4_d3.traj'
# name = 'Cu_100_u2_no14_CO_n3_d3.traj'
# name = 'Cu_100_u2_no05_CO_n3_d3.traj'
# name = 'Ag_100_u2_no14_CO_n3_d3.traj'
# name = 'Ag_100_u2_no21_CO_n2_d3.traj'

allname = name[:-5] + '_all.traj'
atoms = query(name, env)
# query(allname, env)

In [4]:
repeat = 3 # repeat
ratoms = get_repeated_atoms(atoms, repeat)

b_mat, nads = get_bonding_matrix(ratoms)

results = get_number_matrix(b_mat, nads, repeat)
results

array([[0, 2, 6],
       [1, 2, 6],
       [2, 2, 6],
       [3, 2, 6]])

In [24]:
def __get_coordination_string_mod(nn_info):
    '''
    This helper function takes the output of the `VoronoiNN.get_nn_info` method
    and gives you a standardized coordination string.
    Arg:
        nn_info     The output of the
                    `pymatgen.analysis.local_env.VoronoiNN.get_nn_info` method.
    Returns:
        coordination    A string indicating the coordination of the site
                        you fed implicitly through the argument, e.g., 'Cu-Cu-Cu'
    '''
    coordinated_atoms = [neighbor_info['site'].species_string
                         for neighbor_info in nn_info
                         if neighbor_info['site'].species_string != 'Kr']
    coordination = '-'.join(sorted(coordinated_atoms))
    
    coordinated_indexes = [neighbor_info['site_index']
                         for neighbor_info in nn_info
                         if neighbor_info['site'].species_string != 'Kr']

    return coordination, coordinated_indexes

def get_modified_bonding_matrix(atoms):
    '''
    This function will fingerprint a slab+adsorbate atoms object for you.
    It only  with multiple adsorbates.
    Arg:
        atoms   `ase.Atoms` object to fingerprint. The slab atoms must be
                tagged with non-zero integers and adsorbate atoms must be
                tagged with zero. This function also assumes that the
                first atom in each adsorbate is the binding atom (e.g.,
                of all atoms with tag==1, the first atom is the binding;
                the same goes for tag==2 and tag==3 etc.).
    Returns:
        fingerprint A dictionary whose keys are:
                        coordination            A string indicating the
                                                first shell of
                                                coordinated atoms
                        neighborcoord           A list of strings
                                                indicating the coordination
                                                of each of the atoms in
                                                the first shell of
                                                coordinated atoms
                        nextnearestcoordination A string identifying the
                                                coordination of the
                                                adsorbate when using a
                                                loose tolerance for
                                                identifying "neighbors"
    '''
    # Replace the adsorbate[s] with a single Krypton atom at the first binding
    # site. We need the Krypton there so that pymatgen can find its
    # coordination.  
    atoms, binding_positions = removemolecule(atoms, 'CO')
    nads = len(binding_positions)
    for i in reversed(range(nads)):
        atoms += Atoms('Kr', positions=[binding_positions[i]])
    b_mat = np.zeros([len(atoms), len(atoms)])
    Krypton_indexes = []
    for atom in atoms:
        if atom.symbol == 'Kr':
            Krypton_indexes.append(atom.index)
    struct = AseAtomsAdaptor.get_structure(atoms)
    
    try:
        for atom in atoms:
            # We have a standard and a loose Voronoi neighbor finder for various
            # purposes
            vnn = VoronoiNN(allow_pathological=True, tol=0.6, cutoff=10) # originally tol=0.8
            vnn_loose = VoronoiNN(allow_pathological=True, tol=0.2, cutoff=10)

            # Find the coordination
            if atom.symbol == 'Kr':
                nn_info = vnn.get_nn_info(struct, n=atom.index)
                coordination, cindexes = __get_coordination_string_mod(nn_info)
            else:
                nn_info = vnn_loose.get_nn_info(struct, n=atom.index)
                coordination, cindexes = __get_coordination_string_mod(nn_info)
            for cindex in cindexes:
                b_mat[atom.index][cindex] = 1/len(cindexes)
                b_mat[cindex][atom.index] = 1/len(cindexes)
            
        return b_mat, nads
        
    # If we get some QHull or ValueError, then just assume that the adsorbate desorbed
    except (QhullError, ValueError):
        return None
    
def get_modified_number_matrix(b_mat, nads, repeat):
#     b_matlis = []
#     sumb_matlis = []
    newb_mat = copy.deepcopy(b_mat)
    sumb_mat = 0
    results = []

    for i in range(2,6):
        newb_mat = newb_mat @ b_mat
#         b_matlis.append(newb_mat)
        sumb_mat = sumb_mat + newb_mat
#         sumb_matlis.append(sumb_mat)
        sumb_matCO = (sumb_mat - np.diag(np.diag(sumb_mat)))[-nads:,-nads:] # extract related adsorbate and non-diagonal terms
        sumb_matCO = sumb_matCO[nads//repeat**2*4:nads//repeat**2*5] # examinimg only center one is sufficient

    for j in range(nads//repeat**2):
        nnearestbonding = np.max(np.sum(sumb_matCO[j:nads:nads//repeat**2,:], axis=1))
#                     print('adsorbate {} has {} nearest adsorbate at {}th neighbor'.format(j,nnearestbonding,i))
#                     print(newb_matCO[j:nads:nads//repeat**2,:])
        results.append([j,i,nnearestbonding])

    return np.array(results) # [adsorbate_index, distance, # of nearest adsorbate]

In [25]:
name = 'Au_111_u2_no021_CO_n4_d3.traj'
name = 'Au_111_u2_no006_CO_n4_d3.traj'
name = 'Au_111_u2_no004_CO_n4_d3.traj'
name = 'Au_111_u2_no030_CO_n4_d3.traj'
atoms = query(name)

repeat = 3 # repeat
ratoms = get_repeated_atoms(atoms, repeat)

b_mat, nads = get_modified_bonding_matrix(ratoms)

results = get_modified_number_matrix(b_mat, nads, repeat)
results

array([[0.        , 5.        , 4.24691358],
       [1.        , 5.        , 4.24691358],
       [2.        , 5.        , 4.24691358],
       [3.        , 5.        , 4.24691358]])

In [16]:
np.sum((b_mat@b_mat)[-nads,-nads:])

1.0

In [ ]:
array([[0.        , 2.        , 0.66666667],
       [1.        , 2.        , 0.66666667],
       [2.        , 2.        , 0.66666667],
       [3.        , 2.        , 0.66666667],
       [0.        , 3.        , 0.59259259],
       [1.        , 3.        , 0.59259259],
       [2.        , 3.        , 0.59259259],
       [3.        , 3.        , 0.59259259]])